# Sistemas de Recomendação - Razzle Dazzle

Durante a pandemia os números de vendas da Razzle Dazzle (e-commerce de variedades) dispararam, bem como o da concorrência. 

Uma das maneiras de se destacar nesse mercado cada vez mais competitivo é oferecer o produto certo para a pessoa certa.


Portanto existe a necessidade de desenvolver um modelo de recomendação, que será disponibilizado no novo site da empresa. 

Além do modelo, é necessário informar qual o melhor momento para usá-lo, em propagandas, quando o cliente estiver fazendo uma pesquisa ou quando o cliente estiver vendo um produto. É fundamental que isso seja pensado no desenvolvimento do modelo.

Bases de dados fornecidas pela empresa: 
* Orders
* Orders Itens
* Orders Reviews
* Products

## Tratamento inicial

### Importando os pacotes necessários e configuração

In [43]:
from google.colab import files

import progressbar as pb
import warnings # supress warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format

### Fazendo importação dos datasets

Os datasets foram disponibilizados em arquivos .csv

In [2]:
# Upload do arquivo
uploaded = files.upload()

KeyboardInterrupt: ignored

### Carregando dados

Ler o arquivo csv e colocar em um dataframe.

In [2]:
orders_df = pd.read_csv('DNC_orders_dataset.csv')
orders_itens_df = pd.read_csv('DNC_order_items_dataset.csv')
orders_reviews_df = pd.read_csv('DNC_order_reviews_dataset.csv')
products_df = pd.read_csv('DNC_products_dataset.csv')

### Analisando os dados brevemente

O objetivo desta exploração é entender a estrutura dos datasets e avaliar as necessidades de limpeza dos dados para nosso objetivo

In [3]:
def analiseInicial(dataf, desc):
  display(dataf.head().style.set_caption('Sample of ' + desc))
  print(f'Shape is {dataf.shape} \n\n')

  # Avaliando valores null
  print(f'Avaliando nulls\n\n{dataf.isnull().sum()}\n\n')

  # Describe
  print(f'Describe de todas as variáveis\n\n {dataf.describe()}')

In [4]:
analiseInicial(orders_df, 'orders_df')
analiseInicial(orders_itens_df, 'orders_itens_df')
analiseInicial(orders_reviews_df, 'orders_reviews_df')
analiseInicial(products_df, 'products_df')

,Unnamed: 0,order_id,customer_id,order_status
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered


Shape is (99441, 4) 


Avaliando nulls

Unnamed: 0      0
order_id        0
customer_id     0
order_status    0
dtype: int64


Describe de todas as variáveis

        Unnamed: 0
count   99441.000
mean    49720.000
std     28706.288
min         0.000
25%     24860.000
50%     49720.000
75%     74580.000
max     99440.000


,Unnamed: 0,order_id,order_item_id,product_id,price
0,0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.900000
1,1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.900000
2,2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.000000
3,3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.990000
4,4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.900000


Shape is (112650, 5) 


Avaliando nulls

Unnamed: 0       0
order_id         0
order_item_id    0
product_id       0
price            0
dtype: int64


Describe de todas as variáveis

        Unnamed: 0  order_item_id      price
count  112650.000     112650.000 112650.000
mean    56324.500          1.198    120.654
std     32519.398          0.705    183.634
min         0.000          1.000      0.850
25%     28162.250          1.000     39.900
50%     56324.500          1.000     74.990
75%     84486.750          1.000    134.900
max    112649.000         21.000   6735.000


,Unnamed: 0,review_id,order_id,review_score
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


Shape is (99224, 4) 


Avaliando nulls

Unnamed: 0      0
review_id       0
order_id        0
review_score    0
dtype: int64


Describe de todas as variáveis

        Unnamed: 0  review_score
count   99224.000     99224.000
mean    49611.500         4.086
std     28643.646         1.348
min         0.000         1.000
25%     24805.750         4.000
50%     49611.500         5.000
75%     74417.250         5.000
max     99223.000         5.000


,Unnamed: 0,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.000000,287.000000,225.000000,16.000000,10.000000,14.000000
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.000000,276.000000,1000.000000,30.000000,18.000000,20.000000
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.000000,250.000000,154.000000,18.000000,9.000000,15.000000
3,3,cef67bcfe19066a932b7673e239eb23d,bebes,27.000000,261.000000,371.000000,26.000000,4.000000,26.000000
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.000000,402.000000,625.000000,20.000000,17.000000,13.000000


Shape is (32951, 9) 


Avaliando nulls

Unnamed: 0                      0
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64


Describe de todas as variáveis

        Unnamed: 0  product_name_lenght  ...  product_height_cm  product_width_cm
count   32951.000            32341.000  ...          32949.000         32949.000
mean    16475.000               48.477  ...             16.938            23.197
std      9512.279               10.246  ...             13.638            12.079
min         0.000                5.000  ...              2.000             6.000
25%      8237.500               42.000  ...              8.000            15.000
50%     16475.000               51.000  ...             13.000            20.000
75%     24712.500               57.000 

### Limpeza dos dados

O objetivo desta etapa é tratar as bases de dados para deixar as mesmas prontas para análises

In [5]:
def limpezaDados(dataf,desc):
  try:
    # Drop da coluna Unnamed: 0	de todos os datasets
    dataf.drop(['Unnamed: 0'], axis=1, inplace=True)
    print('Coluna Unnamed:0 excluída do df ' + desc)
  except:
    print('Não existe coluna Unnamed:0 para excluir no df ' + desc)
  finally:
    # Como existem poucos valores nulos, irei excluir todas as linhas que tenha alguma informação faltante
    dataf.dropna()

In [6]:
limpezaDados(orders_df,'orders_df')
limpezaDados(orders_itens_df,'orders_itens_df')
limpezaDados(orders_reviews_df,'orders_reviews_df')
limpezaDados(products_df,'products_df')

Coluna Unnamed:0 excluída do df orders_df
Coluna Unnamed:0 excluída do df orders_itens_df
Coluna Unnamed:0 excluída do df orders_reviews_df
Coluna Unnamed:0 excluída do df products_df


### Exploração dos Dados

O objetivo desta etapa é explorar os dados e identificar:
* Algumas correlações
* Como as tabelas se relacionam

In [7]:
orders_df.sample(3)

,order_id,customer_id,order_status
39591,9344b511665d7cc933ea273f0231bf61,9084a4f87533ee359f52a7fb7673e880,delivered
61278,61a689b9ff547322cd08d17069939e86,dafde71460a8cb1b3c205fc8825cdc53,delivered
1132,7032fb16c3a9d982c9aa86f47cefdd3e,d946aedc7ae05f2259b2c2f9342e6272,delivered


In [8]:
orders_itens_df.sample(3)

,order_id,order_item_id,product_id,price
81635,b9895a352437ec0701eccabb55cc43ea,1,4955b9229e73cd2a3344be9cd8ad0d5f,119.900
109329,f86f6472fa9f395d8d9a6751ca118afb,1,99f265229d46f700208ad7cb1ff48aae,44.990
90982,cea4910391a77b0022ce76f53574a0b3,1,57eea82fdaafa70e1bda51d338583f9b,98.900


In [9]:
products_df.sample(3)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
16986,c1254983bd81d72b96867b6838582cf5,esporte_lazer,46.000,565.000,2050.000,20.000,20.000,20.000
16595,9663cf1ff5c7ff603be0a27b1cf2dde2,bebes,44.000,271.000,600.000,25.000,15.000,25.000
25430,e7f79c79a485447b88964a9d0b86fe45,perfumaria,45.000,971.000,67.000,16.000,12.000,14.000


In [10]:
orders_reviews_df.sample(3)

,review_id,order_id,review_score
11586,21d11eb56acf674dc3d5e0177979ddca,13f13b57aacaec347d9d8f974528cf62,5
20646,0c76e7a547a531e7bf9f0b99cba071c1,3525e0e57f9d276d522d570bd46cb39c,5
98619,f486cd11667348d73dae1d150fba5dbc,9887daa2e1181336df4be8ea011d196a,5


In [11]:
# Criando dataframe com os joins
dfCompilado = pd.merge(orders_itens_df, products_df, how='left', on='product_id')
dfCompilado = pd.merge(dfCompilado,orders_df, how='left', on='order_id')
dfCompilado = pd.merge(dfCompilado,orders_reviews_df, how='left', on='order_id')
dfCompilado.drop(['order_item_id','review_id','product_name_lenght', 'product_description_lenght', 'product_length_cm', 'product_height_cm','product_width_cm','product_weight_g'], axis=1, inplace=True)
#reordenar colunas
dfCompilado = dfCompilado[['customer_id','order_id','order_status','product_id','product_category_name','price','review_score']]


In [12]:
dfCompilado.sample(5)

,customer_id,order_id,order_status,product_id,product_category_name,price,review_score
77450,4ef4f77528c601288dcfb9c522fcf103,af4812c0286cb623517321205ffb5d1e,delivered,48e169d7e70bdfd06b008415e0f072e3,construcao_ferramentas_construcao,6.100,5.000
10141,556910dd99aff6bc666910e9a872ec6b,17220bb11a8bc3ddfe388dedfe4e4de9,delivered,eed87e4310f2dbbddf043552065b660e,fashion_bolsas_e_acessorios,24.900,5.000
99065,695ad68c4f5db80c7281c0040e623369,df5db2ddf13eb709a018dea459afb4b5,delivered,4e4210d7d638f6e904297071820f08db,relogios_presentes,26.900,3.000
46893,2cfbc515734b19a3107a9e5b0cd7b250,69eb180c9de9dcddefe8a3e8a7ce8016,delivered,cd9d71c495bc54dbb324571016b7f37c,moveis_escritorio,159.940,5.000
78228,9b04a56f34a9db495868ffb6033fe2f5,b0fd0236ce757738793836930b830f1e,delivered,8c154410716f31649d25e436f7d5d838,beleza_saude,95.000,5.000


In [67]:
dfCompilado.to_csv('dfCompilado.csv', index = False, header=True) 

In [13]:
dfCompilado['review_score'].value_counts()

5.000    63525
4.000    21315
1.000    14235
3.000     9423
2.000     3874
Name: review_score, dtype: int64

In [14]:
dfCompilado['order_status'].value_counts()

delivered      110840
shipped          1196
canceled          546
invoiced          364
processing        358
unavailable         7
approved            3
Name: order_status, dtype: int64

In [15]:
dfCompilado['product_category_name'].value_counts()

cama_mesa_banho                  11270
beleza_saude                      9727
esporte_lazer                     8700
moveis_decoracao                  8415
informatica_acessorios            7894
                                 ...  
cds_dvds_musicais                   14
la_cuisine                          14
pc_gamer                             9
fashion_roupa_infanto_juvenil        8
seguros_e_servicos                   2
Name: product_category_name, Length: 73, dtype: int64

In [42]:
dfCompilado[dfCompilado['order_status']!='delivered']['review_score'].value_counts()

1.000    1660
5.000     213
3.000     181
2.000     174
4.000     131
Name: review_score, dtype: int64

## Recomendações

### Recomendação de itens mais/melhores avaliados

Essa recomendação pode ser utilizada quando um cliente entrar no site e sair sem fazer compra alguma

In [62]:
ratings_explicit = dfCompilado[(dfCompilado['review_score'] != 0) & (dfCompilado['order_status'] == 'delivered')]

ratings_sum = pd.DataFrame(dfCompilado.groupby(['product_id'])['review_score'].sum())\
  .sort_values('review_score', ascending=False)

ratings_sum.head(10)

,review_score
product_id,
aca2eb7d00ea1a7b8ebd4e68314663af,2106.000
422879e10f46682990de24d770e7f83d,1918.000
99a4788cb24856965c36a24e339b6058,1879.000
389d119b48cf3043d311335e499d9c6b,1610.000
368c6c730842d78016ad823897a372db,1522.000
53759a2ecddad2bb87a079a1f1519f73,1443.000
d1c427060a0f73f6b889a5c7c61f2ac4,1426.000
53b36df67ebb7c41585e8d54d6772e08,1341.000
154e7e31ebfa092203795c972e5804a6,1260.000


### Recomendação de itens mais populares (mais comprados)

Essa recomendação pode ser utilizada quando um cliente entrar no site e sair sem fazer compra alguma

In [64]:
count = dfCompilado.groupby(by='product_id', as_index=False) \
  .agg({'customer_id': pd.Series.nunique}) \
  .rename(columns={'customer_id': 'customer_count'})

buy_count = count.set_index('product_id') \
  .sort_values('customer_count', ascending = False)

buy_count.head(10)

,customer_count
product_id,
99a4788cb24856965c36a24e339b6058,467
aca2eb7d00ea1a7b8ebd4e68314663af,431
422879e10f46682990de24d770e7f83d,352
d1c427060a0f73f6b889a5c7c61f2ac4,323
389d119b48cf3043d311335e499d9c6b,311
53b36df67ebb7c41585e8d54d6772e08,306
368c6c730842d78016ad823897a372db,291
53759a2ecddad2bb87a079a1f1519f73,287
154e7e31ebfa092203795c972e5804a6,269


### sss

In [82]:
%%writefile data.py
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def get_data(path):
  return pd.read_csv(path)


def start_pipeline(dataf):
  return dataf.copy()


def get_book_counts(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'customer_id': pd.Series.nunique}) \
    .rename(columns={'customer_id': 'customer_id_count'}) \
    .set_index('product_id') \
    .sort_values('customer_id_count', ascending = False)


def get_n_top_values(dataf, n):
  return dataf.head(n) \
    .index \
    .tolist()


def format_sample(dataf):
  return pd.DataFrame({
      "user": dataf["customer_id"],
      "book": dataf["product_id"],
      "action": dataf["review_score"]
  })


def get_top_books_ids(dataf, sample_size=1000):
  return dataf \
    .pipe(start_pipeline) \
    .pipe(get_book_counts) \
    .pipe(get_n_top_values, sample_size)


def get_item_based_similarity_matrix(data):
  top_book_ids = get_top_books_ids(data)
  book_sampled_relationship_data = data[data["product_id"].isin(top_book_ids)] \
    .pipe(format_sample) \
    .pivot_table(index="user", columns="book", values="action").fillna(0)
  return cosine_similarity(book_sampled_relationship_data.T), book_sampled_relationship_data.T.index

Overwriting data.py


In [83]:
%%writefile item_based_recommender.py
import random
import numpy as np

from data import get_data, get_item_based_similarity_matrix

class ItemBasedRecommender:

  def __init__(self, data_path):
    self.data_path = data_path

  def setup(self):
    data = get_data(self.data_path)
    sim_matrix, available_items_index = get_item_based_similarity_matrix(data)
    self.available_items_index = available_items_index
    self.sim_matrix = sim_matrix

  def get_random_item_index(self):
    item_id = self.available_items_index[random.randint(0, self.sim_matrix.shape[0])]
    return item_id

  def get_n_closest(self, item_id, n=10):
    if item_id not in self.available_items_index:
      raise ValueError('Item is not available on data.')
    item_index = self.available_items_index.tolist().index(item_id)
    closest_n_iten = np.argsort(-self.sim_matrix[item_index])[1:n]
    n_closest = []
    for item_data in zip(self.available_items_index[closest_n_iten], self.sim_matrix[item_index][closest_n_iten]):
      n_closest.append(item_data)
    return tuple(n_closest)

Overwriting item_based_recommender.py


In [84]:
from item_based_recommender import ItemBasedRecommender

ratings_df_path = 'dfCompilado.csv'

item_recommender = ItemBasedRecommender(ratings_df_path)

item_recommender.setup()

NameError: ignored